In [0]:
SELECT
  feedback_id,      -- フィードバックID
  user_id,          -- 会員ID
  product_id,       -- 商品ID
  rating,           -- 評価
  date,             -- フィードバック日
  -- カテゴリ
  ai_query(
    'databricks-claude-3-7-sonnet',
    CONCAT(
      "[指示]次の顧客レビュー内容を、次のカテゴリのいずれかに分類してください",
      "\n[顧客レビュー]" || comment,
      "\n[分類カテゴリ]",
      "\n品揃え・在庫",
      "\n品質",
      "\nコスパ・お得感",
      "\n接客・サービス",
      "\n店舗設備・環境",
      "\nその他",
      "\n[厳守事項]",
      "\n * カテゴリから1つのみ選択してください。",
      "\n * カテゴリのみ出力してください。補足は一切不要です。"
    ), failOnError => False
  ).result AS category,
  -- 要約
  ai_query(
    'databricks-claude-3-7-sonnet',
    CONCAT(
      "[指示]次の顧客レビューから要点を絞って、明快で簡潔な一つの文章に要約して下さい",
      "\n[顧客レビュー]" || comment,
      "\n[厳守事項]",
      "\n * 要約結果のみ出力してください。こちらの指示に関する補足は一切不要です。",
      "\n * 30文字以内に収めること。"
    ), failOnError => False
  ).result AS summary,
  -- ポジティブスコア
  CAST( 
    ai_query(
        'databricks-claude-3-7-sonnet',
        CONCAT(
        "[指示]次の顧客レビュー内容について、0~1の間でポジティブスコアを付与してください",
        "\n[顧客レビュー]" || comment,
        "\n[出力形式]",
        "\n * 範囲0~1の少数第二位までで出力してください。",
        "\n * 1に近いほどポジティブ度合いが強く、0に近いほどポジティブ度合いが弱い。",
        "\n[厳守事項]",
        "\n * ポジティブスコアのみ出力してください。補足は一切不要です。"
        ), failOnError => False
    ).result AS FLOAT) AS positive_score,
  comment
FROM komae_demo_v3.bricksmart.feedbacks                          -- TODO: カタログ名をご自身の環境に合わせて変更ください
LIMIT 5;
